In [18]:
import requests
import time
from tqdm import trange
import pandas as pd
import re

In [ ]:
# 예시
url = f'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1&start=0&length=100'
response = requests.get(url)
print("response = ", response)
data_json = response.json()
print("data_json = ",data_json)
records_total = data_json["recordsTotal"]

data = data_json["data"]

In [5]:
if (records_total % 100) == 0:
  end_page = records_total // 100
else:
  end_page = (records_total // 100) + 1

print("end_page", end_page)

end_page 297


In [12]:
def get_seoul_covid19(page_no):
  start_no = (page_no - 1) * 100

  url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}&start={start_no}&length=100"

  response = requests.get(url)
  # print("page_no = ",page_no)

  data_json = response.json()
  return data_json


In [ ]:
patient_list = []

for page_no in trange(1, end_page + 1):
  one_page = get_seoul_covid19(page_no)
  patient = pd.DataFrame(one_page["data"])
  print("patient = ", patient)
  print("=" * 100)

  patient_list.append(patient)
  time.sleep(0.5)

In [ ]:
df_all = pd.concat(patient_list)
print("df_all = ", df_all)
df_all.columns =  ["발생번호", "환자번호", "확진일", "거주지", "여행력", "접촉력", "퇴원현황"]
print("df_all_new", df_all)

In [ ]:
def extract_number(num_string):
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "")
        num = re.sub("[^0-9]","", num_string)

        num =int(num)
        return num
    else:
        return num_string

df_all["발생번호"] = df_all["발생번호"].map(extract_number)

In [20]:
def extract_hangeul(origin_text):
    substract_text = re.sub("[^가-힣]", "", origin_text)
    return substract_text

df_all["퇴원현황"] = df_all["퇴원현황"].map(extract_hangeul)

print("df_all['퇴원현황'] = ", df_all['퇴원현황'])

df_all.to_csv("seoul_covid19.csv" , index = False)

df_all['퇴원현황'] =  0       
1       
2       
3       
4       
      ..
39    퇴원
40    퇴원
41    퇴원
42    퇴원
43    퇴원
Name: 퇴원현황, Length: 29644, dtype: object
